In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import sklearn.metrics as metrics
import sklearn.cluster as cluster
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [ ]:
df = pd.read_csv('/content/preprocessed_data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

#Pré - processamento

##Dados Faltantes

In [ ]:
df.info()

In [ ]:
df.select_dtypes(include= 'object').describe()

##Transformando as variaveis categoricas

##Category

In [ ]:
df['category'].unique()

In [ ]:
df['category']= df.category.str.split('|')
df['category']  = df['category'].str[0]

In [ ]:
df['category'].unique()

##Marca

In [ ]:
df.brand.nunique()
#Valores diferentes

In [ ]:
df['brand'].unique()

In [ ]:
df_encoded = pd.get_dummies(df[['category','brand']])


df_final = pd.concat([df, df_encoded], axis=1)
df_final.drop(['brand', 'category','subcategory','Unnamed: 0','event_time','event_type','category_code','user_session','product_id','category_id','user_id'], axis=1, inplace=True)
df_final

In [ ]:
df_final.columns

In [ ]:
df_final.info()

In [ ]:
X= df_final.values
X

#Padronização dos dados

In [ ]:
# Padronizando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)


#Utilização PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2 ).fit_transform(X)

#Metodo Cotovelo

In [ ]:
n_clusters = range(1, 11)
cluster_sse = []
for i in n_clusters:
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(pca)
    cluster_sse.append(kmeans.inertia_)

plt.plot(n_clusters, cluster_sse, marker = "o")
plt.show()

In [ ]:
modelo_v1 = KMeans(n_clusters = 4,random_state=42)
modelo_v1.fit(pca)

In [ ]:
y_kmeans = modelo_v1.predict(pca)

In [ ]:
plt.figure(figsize=(8, 6))

# Plotando os resultados
scatter = plt.scatter(pca[:, 0], pca[:, 1], c=y_kmeans, s=50, cmap='viridis')

# Plotando os centróides dos clusters
centers = modelo_v1.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5, label='Centroides')

cluster_labels = {0: 'Cluster 0', 1: 'Cluster 1', 2: 'Cluster 2', 3: 'Cluster 3'}
plt.legend(handles=scatter.legend_elements()[0], labels=[cluster_labels[i] for i in range(len(cluster_labels))], title='Cluster')

plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('PCA com K-Means')
plt.show()

#KMeans

In [ ]:
#Aplicando o Kmeans
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X)

In [ ]:
x_kmeans = kmeans.predict(X)

In [ ]:
plt.figure(figsize=(8, 6))

# Plotando os resultados
scatter = plt.scatter(X[:, 0], X[:, 1], c=x_kmeans, s=50, cmap='viridis')

# Plotando os centróides dos clusters
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5, label='Centroides')

cluster_labels = {0: 'Cluster 0', 1: 'Cluster 1', 2: 'Cluster 2', 3: 'Cluster 3'}
plt.legend(handles=scatter.legend_elements()[0], labels=[cluster_labels[i] for i in range(len(cluster_labels))], title='Cluster')

plt.xlabel('Features 1 ')
plt.ylabel('Features 2 ')
plt.title('Cluster com K-Means')
plt.show()


##Validação dos Cluster

O índice Calinski-Harabasz, também conhecido como Critério de Razão de Variância, é a razão entre a soma dedispersão entre clusterse dedispersão entre clusters para todos os clusters, quanto maior a pontuação, melhores serão os desempenhos.

In [ ]:
previsoes = modelo_v1.predict(pca)
#Calinski-Harabasz Score
calinski_harabasz = calinski_harabasz_score(pca, previsoes)
print("Calinski-Harabasz Score: {:.2f}".format(calinski_harabasz))


In [ ]:
previsoes = modelo_v1.predict(pca)

# Calcule o Índice Davies-Bouldin
db_score = davies_bouldin_score(pca, previsoes)

print("Índice Davies-Bouldin: {:.4f}".format(db_score))
